# Implementation of Graph Convolutional Neural Networks (Vanilla)

Based upon:
- **Paper:** Kipf & Welling (ICLR 2017), Semi-Supervised Classification with Graph Convolutional Networks, [https://arxiv.org/pdf/1609.02907.pdf](https://arxiv.org/pdf/1609.02907.pdf)

- **Blog Post:** [https://tkipf.github.io/graph-convolutional-networks/](https://tkipf.github.io/graph-convolutional-networks/)

- **Implementation:** https://github.com/rusty1s/pytorch_geometric/blob/master/examples/gcn.py


In [1]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.5MB 417kB/s 
     |████████████████████████████████| 1.5MB 379kB/s 
     |████████████████████████████████| 1.0MB 3.5MB/s 
     |████████████████████████████████| 215kB 2.9MB/s 
     |████████████████████████████████| 235kB 4.9MB/s 
     |████████████████████████████████| 2.2MB 5.6MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv
import os.path as osp

## Load Dataset

In [10]:
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


## GCN Model

The **GCN layer** ([Kipf et al. (2017)](https://arxiv.org/abs/1609.02907)) is defined as

$$
\mathbf{x}_v^{(\ell + 1)} = \mathbf{W}^{(\ell + 1)} \sum_{w \in \mathcal{N}(v) \, \cup \, \{ v \}} \frac{1}{c_{w,v}} \cdot \mathbf{x}_w^{(\ell)}
$$

where $\mathbf{W}^{(\ell + 1)}$ denotes a trainable weight matrix of shape `[num_output_features, num_input_features]` and $c_{w,v}$ refers to a fixed normalization coefficient for each edge.

In [11]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, dropout):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

## Train Model on Cora Dataset

In [15]:
args = {
    "device" : 'cuda' if torch.cuda.is_available() else 'cpu',
    "epochs" : 150,
    "lr" : 0.01,
    "weight_decay" : 5e-4,
    "hidden" : 16,
    "dropout": 0.5
}

In [14]:
model = GCN(hidden_channels=args["hidden"], dropout=args["dropout"]).to(args["device"])
data = data.to(args["device"])
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [16]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [21]:
def train(epoch):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss_train = F.nll_loss(out[data.train_mask], data.y[data.train_mask]) # Compute the loss solely based on the training nodes.
      acc_train = accuracy(out[data.train_mask], data.y[data.train_mask])
      loss_train.backward()  # Derive gradients.
      optimizer.step()

      model.eval()
      out = model(data.x, data.edge_index)

      loss_val = F.nll_loss(out[data.val_mask], data.y[data.val_mask])
      acc_val = accuracy(out[data.val_mask], data.y[data.val_mask])

      if epoch % 10 == 0:
        print('Epoch: {:03d}'.format(epoch),
              'loss_train: {:.3f}'.format(loss_train.item()),
              'acc_train: {:.3f}'.format(acc_train.item()),
              'loss_val: {:.3f}'.format(loss_val.item()),
              'acc_val: {:.3f}'.format(acc_val.item()))

In [24]:
def test():      
    model.eval()
    output = model(data.x, data.edge_index)
    loss_test = F.nll_loss(output[data.test_mask], data.y[data.test_mask])
    acc_test = accuracy(output[data.test_mask], data.y[data.test_mask])
    print("Test set results:",
          "loss = {:.3f}".format(loss_test.item()),
          "accuracy = {:.3f}".format(acc_test.item()),
          f" = {round(acc_test.item() * 100, 3)}%")

In [25]:
# Train model
for epoch in range(args["epochs"] + 1):
    train(epoch)
print("Optimization Finished!")

# Testing
test()

Epoch: 000 loss_train: 0.382 acc_train: 0.979 loss_val: 0.866 acc_val: 0.794
Epoch: 010 loss_train: 0.369 acc_train: 0.979 loss_val: 0.843 acc_val: 0.790
Epoch: 020 loss_train: 0.347 acc_train: 0.986 loss_val: 0.827 acc_val: 0.796
Epoch: 030 loss_train: 0.340 acc_train: 0.986 loss_val: 0.818 acc_val: 0.798
Epoch: 040 loss_train: 0.311 acc_train: 0.979 loss_val: 0.809 acc_val: 0.794
Epoch: 050 loss_train: 0.315 acc_train: 0.950 loss_val: 0.805 acc_val: 0.794
Epoch: 060 loss_train: 0.266 acc_train: 0.993 loss_val: 0.801 acc_val: 0.788
Epoch: 070 loss_train: 0.280 acc_train: 0.979 loss_val: 0.797 acc_val: 0.792
Epoch: 080 loss_train: 0.271 acc_train: 0.979 loss_val: 0.796 acc_val: 0.790
Epoch: 090 loss_train: 0.295 acc_train: 0.971 loss_val: 0.775 acc_val: 0.800
Epoch: 100 loss_train: 0.299 acc_train: 0.986 loss_val: 0.757 acc_val: 0.796
Epoch: 110 loss_train: 0.252 acc_train: 0.971 loss_val: 0.768 acc_val: 0.796
Epoch: 120 loss_train: 0.276 acc_train: 0.986 loss_val: 0.781 acc_val: 0.794